In [11]:
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report
from bs4 import BeautifulSoup

In [12]:
nltk.download('nps_chat')
posts = nltk.corpus.nps_chat.xml_posts()

[nltk_data] Downloading package nps_chat to
[nltk_data]     C:\Users\wangs\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\nps_chat.zip.


In [13]:
def dialogue_act_features(post):
    features = {}
    for word in nltk.word_tokenize(post):
        features['contains({})'.format(word.lower())] = True
    return features

def is_ques_using_nltk(ques):
    question_type = classifier.classify(dialogue_act_features(ques)) 
    return question_type in question_types

question_pattern = ["do i", "do you", "what", "who", "is it", "why","would you", "how","is there",
                    "are there", "is it so", "is this true" ,"to know", "is that true", "are we", "am i", 
                   "question is", "tell me more", "can i", "can we", "tell me", "can you explain",
                   "question","answer", "questions", "answers", "ask"]

helping_verbs = ["is","am","can", "are", "do", "does"]
# check with custom pipeline if still this is a question mark it as a question

def remove_non_ascii(sentence):
    return ''.join(char for char in sentence if ord(char) < 128)
    
def html_Filter(sentence):
    sentence = BeautifulSoup(sentence, "lxml").text
    #print("after html_Filter",sentence)
    
    return sentence

def is_question(question):
    question = question.lower().strip()
    question = remove_non_ascii(question)
    question = html_Filter(question)
    
    if not is_ques_using_nltk(question):
        is_ques = False
        # check if any of pattern exist in sentence
        for pattern in question_pattern:
            is_ques  = pattern in question
            if is_ques:
                break

        # there could be multiple sentences so divide the sentence
        sentence_arr = question.split(".")
        for sentence in sentence_arr:
            if len(sentence.strip()):
                # if question ends with ? or start with any helping verb
                # word_tokenize will strip by default
                first_word = nltk.word_tokenize(sentence)[0]
                if sentence.endswith("?") or first_word in helping_verbs:
                    is_ques = True
                    break
        return is_ques    
    else:
        return True

In [14]:


featuresets = [(dialogue_act_features(post.text), post.get('class')) for post in posts]

# 10% of the total data
size = int(len(featuresets) * 0.1)

# first 10% for test_set to check the accuracy, and rest 90% after the first 10% for training
train_set, test_set = featuresets[size:], featuresets[:size]

# get the classifer from the training set
classifier = nltk.NaiveBayesClassifier.train(train_set)
# to check the accuracy - 0.67
# print(nltk.classify.accuracy(classifier, test_set))

question_types = ["whQuestion","ynQuestion"]


In [6]:
is_question("Do you have a file named `dlib.py` in the same directory you're trying to import the package from?")

True

In [7]:
is_question("I think it would be yes, but not sure about the OS")

False

In [8]:
is_question("I wonder if the OS can affect to assembly")

False

## Read data

In [1]:
import pandas as pd

In [4]:
df = pd.read_csv("../../ISPY_data/chat_test.csv")
df.fillna("N",inplace=True)

In [5]:
df.head()

,Issues,PS,ETD,TS,AR,Q
0,Hello! I was facing an issue yesterday only wh...,Y,N,N,N,N
1,"Hi, I tried to add the artifactorg.nd4j:nd4j-n...",Y,N,N,N,Y
2,"Hi All,i am facing one issue. Our developers u...",Y,N,N,N,Y
3,"hi all, i am facing some issue with typescript...",Y,N,Y,N,N
4,"Hi, I am using Typescript in a react project, ...",Y,N,N,N,Y


In [15]:
predicted_q = []
for issue in df["Issues"]:
    if(is_question(issue)):
        predicted_q.append("Y")
    else:
        predicted_q.append("N")

In [17]:
df["Q_p"] = predicted_q

In [18]:
df.head()

,Issues,PS,ETD,TS,AR,Q,Q_p
0,Hello! I was facing an issue yesterday only wh...,Y,N,N,N,N,N
1,"Hi, I tried to add the artifactorg.nd4j:nd4j-n...",Y,N,N,N,Y,Y
2,"Hi All,i am facing one issue. Our developers u...",Y,N,N,N,Y,Y
3,"hi all, i am facing some issue with typescript...",Y,N,Y,N,N,N
4,"Hi, I am using Typescript in a react project, ...",Y,N,N,N,Y,Y


In [19]:
df.to_csv("../../ISPY_data/chat_test_result.csv",index=None)